In [ ]:
# pip install pandas numpy matplotlib seaborn openpyxl jupyter scipy

# 1. 통계 분석

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

colors = {'primary': '#1a5276', 'secondary': '#2874a6', 'accent': '#e74c3c', 
          'light': '#85c1e9', 'gray': '#7f8c8d', 'green': '#27ae60'}

# 데이터 로드
sales = pd.read_csv('sales_convenience.csv', encoding='utf-8-sig')
population = pd.read_csv('population.csv', encoding='utf-8-sig')

pop_cols = ['기준_년분기_코드', '행정동_코드', '총_유동인구_수',
            '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수',
            '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수']

merged = sales.merge(population[pop_cols], on=['기준_년분기_코드', '행정동_코드'], how='inner')



In [2]:
# 변수 생성
merged['유동인구'] = merged['총_유동인구_수']
merged['매출금액'] = merged['당월_매출_금액']
merged['매출건수'] = merged['당월_매출_건수']
merged['밀집도'] = (merged['매출건수'] / merged['유동인구']) * 1000

merged['청년_유동인구_비율'] = (merged['연령대_20_유동인구_수'] + merged['연령대_30_유동인구_수']) / merged['유동인구']
merged['중년_유동인구_비율'] = (merged['연령대_40_유동인구_수'] + merged['연령대_50_유동인구_수']) / merged['유동인구']
merged['고령_유동인구_비율'] = merged['연령대_60_이상_유동인구_수'] / merged['유동인구']
merged['10대_유동인구_비율'] = merged['연령대_10_유동인구_수'] / merged['유동인구']

merged['log_매출'] = np.log1p(merged['매출금액'])
merged['log_유동인구'] = np.log1p(merged['유동인구'])
merged['log_밀집도'] = np.log1p(merged['밀집도'])

print("="*80)
print("서울 편의점 매출 분석 (유동인구, 밀집도, 연령대 집중)")
print("="*80)
print(f"\n데이터: {len(merged)}행, {merged['행정동_코드'].nunique()}개 행정동")



서울 편의점 매출 분석 (유동인구, 밀집도, 연령대 집중)

데이터: 6097행, 414개 행정동


In [3]:
# 기술통계
print("\n[기술통계]")
print(f"  밀집도(천명당 매출건수): 평균 {merged['밀집도'].mean():.1f}, 중앙값 {merged['밀집도'].median():.1f}")
print(f"  유동인구: 평균 {merged['유동인구'].mean()/1e6:.2f}백만명")
print(f"  매출: 평균 {merged['매출금액'].mean()/1e8:.1f}억원")




[기술통계]
  밀집도(천명당 매출건수): 평균 76.3, 중앙값 50.5
  유동인구: 평균 5.62백만명
  매출: 평균 27.1억원


In [4]:
# 상관분석
print("\n[상관분석 - log(매출)과의 상관]")
for var in ['log_유동인구', 'log_밀집도', '청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']:
    r, p = stats.pearsonr(merged['log_매출'], merged[var])
    sig = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
    print(f"  {var:<25}: r = {r:.4f} {sig}")




[상관분석 - log(매출)과의 상관]
  log_유동인구                 : r = 0.5137 ***
  log_밀집도                  : r = 0.8139 ***
  청년_유동인구_비율               : r = 0.4492 ***
  중년_유동인구_비율               : r = -0.0351 **
  고령_유동인구_비율               : r = -0.4192 ***


In [5]:
# 회귀분석
Y = merged['log_매출'].values
n = len(Y)
X_vars = ['log_유동인구', 'log_밀집도', '청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']
X_data = merged[X_vars].values
X = np.column_stack([np.ones(n), X_data])

beta = np.linalg.solve(X.T @ X, X.T @ Y)
Y_pred = X @ beta
residuals = Y - Y_pred

SST = np.sum((Y - np.mean(Y))**2)
SSE = np.sum(residuals**2)
k = len(X_vars)
R2 = 1 - SSE / SST
adj_R2 = 1 - (SSE / (n - k - 1)) / (SST / (n - 1))
MSE = SSE / (n - k - 1)
se_beta = np.sqrt(MSE * np.diag(np.linalg.inv(X.T @ X)))
t_stats = beta / se_beta
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n-k-1))

print("\n[회귀분석 결과]")
print(f"  R-squared: {R2:.4f} ({R2*100:.1f}% 설명)")
print(f"  Adjusted R-squared: {adj_R2:.4f}")

print("\n  회귀계수:")
var_names = ['(상수항)'] + X_vars
for i, name in enumerate(var_names):
    sig = "***" if p_values[i] < 0.001 else "**" if p_values[i] < 0.01 else "*" if p_values[i] < 0.05 else ""
    print(f"    {name:<25}: β = {beta[i]:>8.4f}, t = {t_stats[i]:>7.2f} {sig}")

# VIF
def calc_vif(X_data):
    vifs = []
    for i in range(X_data.shape[1]):
        y = X_data[:, i]
        X_o = np.delete(X_data, i, axis=1)
        X_d = np.column_stack([np.ones(len(y)), X_o])
        b = np.linalg.solve(X_d.T @ X_d, X_d.T @ y)
        r2 = 1 - np.sum((y - X_d @ b)**2) / np.sum((y - np.mean(y))**2)
        vifs.append(1 / (1 - r2) if r2 < 1 else float('inf'))
    return vifs

vif_values = calc_vif(X_data)
print("\n[VIF (다중공선성)]")
for var, vif in zip(X_vars, vif_values):
    print(f"    {var:<25}: {vif:.2f}")
print(f"  → 최대 VIF = {max(vif_values):.2f} (5 미만이면 양호)")

# 단독 R2
single_r2 = {}
for var in X_vars:
    X_s = np.column_stack([np.ones(n), merged[var].values])
    b_s = np.linalg.solve(X_s.T @ X_s, X_s.T @ Y)
    r2_s = 1 - np.sum((Y - X_s @ b_s)**2) / SST
    single_r2[var] = r2_s

print("\n[개별 변수 설명력 (R²)]")
for var, r2 in sorted(single_r2.items(), key=lambda x: x[1], reverse=True):
    print(f"    {var:<25}: {r2:.4f}")

# 잔차 진단
dw_stat = np.sum(np.diff(residuals)**2) / np.sum(residuals**2)
print(f"\n[잔차 진단] Durbin-Watson: {dw_stat:.2f}")

# 결과 요약
print("\n" + "="*80)
print("핵심 발견")
print("="*80)
print(f"""
1. 밀집도 효과 (R² = {single_r2['log_밀집도']:.4f}, β = {beta[2]:.4f})
   → 밀집도 1% 증가 시 매출 약 {beta[2]:.2f}% 증가
   → 가장 높은 단독 설명력!

2. 유동인구 효과 (R² = {single_r2['log_유동인구']:.4f}, β = {beta[1]:.4f})
   → 유동인구 1% 증가 시 매출 약 {beta[1]:.2f}% 증가

3. 연령대 효과
   → 청년(20-30대) 비율: β = {beta[3]:+.4f}
   → 중년(40-50대) 비율: β = {beta[4]:+.4f}
   → 고령(60대+) 비율: β = {beta[5]:+.4f}

4. 다중공선성: 문제 없음 (max VIF = {max(vif_values):.2f})
""")



[회귀분석 결과]
  R-squared: 0.9633 (96.3% 설명)
  Adjusted R-squared: 0.9633

  회귀계수:
    (상수항)                    : β =   0.5641, t =    6.21 ***
    log_유동인구                 : β =   1.0548, t =  209.25 ***
    log_밀집도                  : β =   1.0546, t =  303.22 ***
    청년_유동인구_비율               : β =   0.0763, t =    1.49 
    중년_유동인구_비율               : β =   0.2571, t =    2.46 *
    고령_유동인구_비율               : β =   0.7651, t =   10.41 ***

[VIF (다중공선성)]
    log_유동인구                 : 1.17
    log_밀집도                  : 1.30
    청년_유동인구_비율               : 2.42
    중년_유동인구_비율               : 1.43
    고령_유동인구_비율               : 1.77
  → 최대 VIF = 2.42 (5 미만이면 양호)

[개별 변수 설명력 (R²)]
    log_밀집도                  : 0.6625
    log_유동인구                 : 0.2639
    청년_유동인구_비율               : 0.2018
    고령_유동인구_비율               : 0.1758
    중년_유동인구_비율               : 0.0012

[잔차 진단] Durbin-Watson: 1.90

핵심 발견

1. 밀집도 효과 (R² = 0.6625, β = 1.0546)
   → 밀집도 1% 증가 시 매출 약 1.05% 증가
   → 가장 높은 단독 설명력!

2.

In [6]:

# 결과 저장
results_df = pd.DataFrame({
    '지표': ['R2', 'Adj_R2', 'beta_유동인구', 'beta_밀집도', 'beta_청년', 'beta_중년', 'beta_고령',
             'R2_유동인구', 'R2_밀집도', 'Max_VIF', 'DW'],
    '값': [R2, adj_R2, beta[1], beta[2], beta[3], beta[4], beta[5],
           single_r2['log_유동인구'], single_r2['log_밀집도'], max(vif_values), dw_stat]
})
results_df.to_csv('regression_results_v2.csv', index=False, encoding='utf-8-sig')

merged[['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '매출금액', '유동인구', '매출건수', '밀집도',
        '청년_유동인구_비율', '중년_유동인구_비율', '고령_유동인구_비율']].to_csv(
    'analysis_data_v2.csv', index=False, encoding='utf-8-sig')

print("결과 저장 완료!")

결과 저장 완료!


# 2. 시각화

# 11111111111

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats


In [ ]:
pop = pd.read_csv("population.csv", encoding="utf-8-sig")
sales = pd.read_csv("sales_convenience.csv", encoding="utf-8-sig")
store = pd.read_csv("store_list.csv", encoding="utf-8-sig")


In [ ]:
# 컬럼명 통일
pop = pop.rename(columns={"행정동_코드_명": "행정동명"})
sales = sales.rename(columns={"행정동_코드_명": "행정동명"})

# 편의점만 필터
sales = sales[sales["서비스_업종_코드_명"] == "편의점"]

# 행정동 단위 집계
pop_grp = pop.groupby("행정동명", as_index=False).mean(numeric_only=True)
sales_grp = sales.groupby("행정동명", as_index=False)["당월_매출_금액"].mean()

store_cnt = (
    store.groupby("행정동명")
    .size()
    .reset_index(name="점포수")
)

df = (
    sales_grp
    .merge(pop_grp, on="행정동명", how="left")
    .merge(store_cnt, on="행정동명", how="left")
)

df = df.dropna()


In [ ]:
df["유동인구"] = df["총_유동인구_수"]

df["청년"] = df["연령대_20_유동인구_수"] + df["연령대_30_유동인구_수"]
df["중년"] = df["연령대_40_유동인구_수"] + df["연령대_50_유동인구_수"]
df["고령"] = df["연령대_60_이상_유동인구_수"]

df["청년_비율"] = df["청년"] / df["유동인구"]
df["중년_비율"] = df["중년"] / df["유동인구"]
df["고령_비율"] = df["고령"] / df["유동인구"]

df = df[df["유동인구"] > 0]

df["밀집도"] = df["점포수"] / df["유동인구"]

df["log_매출"] = np.log1p(df["당월_매출_금액"])
df["log_유동인구"] = np.log1p(df["유동인구"])
df["log_밀집도"] = np.log1p(df["밀집도"])


In [ ]:
fig = go.Figure()

def add_scatter(x, y, name, color):
    coef = np.polyfit(x, y, 1)
    x_line = np.linspace(x.min(), x.max(), 100)

    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode="markers",
        opacity=0.4,
        name=name,
        text=df["행정동명"],
        hovertemplate="행정동: %{text}<br>X: %{x:.2f}<br>log(매출): %{y:.2f}"
    ))

    fig.add_trace(go.Scatter(
        x=x_line,
        y=coef[0]*x_line + coef[1],
        mode="lines",
        name=f"{name} (Fit)",
        line=dict(width=3)
    ))

add_scatter(df["log_유동인구"], df["log_매출"], "Floating Population", "blue")
add_scatter(df["log_밀집도"], df["log_매출"], "Store Density", "green")

fig.update_layout(
    title="Sales vs Key Drivers",
    xaxis_title="log(Value)",
    yaxis_title="log(Sales)",
    template="plotly_white"
)

fig.show()


In [ ]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=df["청년_비율"],
    y=df["log_매출"],
    mode="markers",
    opacity=0.3,
    name="Youth (20–30s)"
))

fig2.add_trace(go.Scatter(
    x=df["중년_비율"],
    y=df["log_매출"],
    mode="markers",
    opacity=0.3,
    name="Middle (40–50s)"
))

fig2.add_trace(go.Scatter(
    x=df["고령_비율"],
    y=df["log_매출"],
    mode="markers",
    opacity=0.3,
    name="Senior (60+)"
))

fig2.update_layout(
    title="Age Demographics vs Sales",
    xaxis_title="Population Ratio",
    yaxis_title="log(Sales)",
    template="plotly_white"
)

fig2.show()


In [ ]:
q = 5

df["밀집도_구간"] = pd.qcut(df["밀집도"], q=q, duplicates="drop")
df["유동인구_구간"] = pd.qcut(df["유동인구"], q=q, duplicates="drop")

fig3 = go.Figure(go.Bar(
    x=df.groupby("밀집도_구간")["당월_매출_금액"].mean().index.astype(str),
    y=df.groupby("밀집도_구간")["당월_매출_금액"].mean().values / 1e8
))

fig3.update_layout(
    title="Sales by Store Density Quintile",
    yaxis_title="Avg Sales (100M KRW)",
    template="plotly_white"
)

fig3.show()


In [ ]:
print("밀집도 상관계수:", stats.pearsonr(df["log_밀집도"], df["log_매출"])[0])
print("유동인구 상관계수:", stats.pearsonr(df["log_유동인구"], df["log_매출"])[0])


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go


In [ ]:
def quintile_plot(q):
    temp = df.copy()

    temp["밀집도_구간"] = pd.qcut(temp["밀집도"], q=q, duplicates="drop")

    grp = temp.groupby("밀집도_구간")["당월_매출_금액"].mean() / 1e8

    fig = go.Figure(go.Bar(
        x=grp.index.astype(str),
        y=grp.values
    ))

    fig.update_layout(
        title=f"Sales by Store Density Quintile (q={q})",
        yaxis_title="Avg Sales (100M KRW)",
        template="plotly_white"
    )

    fig.show()


In [ ]:
import folium


In [ ]:
geo = store.groupby("행정동명")[["위도", "경도"]].mean().reset_index()

map_df = df.merge(geo, on="행정동명", how="left").dropna()


In [ ]:
m = folium.Map(
    location=[37.56, 126.97],
    zoom_start=11,
    tiles="cartodbpositron"
)

for _, r in map_df.iterrows():
    folium.CircleMarker(
        location=[r["위도"], r["경도"]],
        radius=5 + np.log1p(r["당월_매출_금액"]) / 2,
        color="blue",
        fill=True,
        fill_opacity=0.6,
        popup=f"""
        <b>{r['행정동명']}</b><br>
        매출: {r['당월_매출_금액'] / 1e8:.2f} 억원<br>
        유동인구: {int(r['유동인구']):,}
        """
    ).add_to(m)

m
